In [2]:
import torch
from torch import nn
from torch.nn import functional as F

In [3]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0818, -0.2376,  0.1310,  0.0181, -0.1299, -0.1497, -0.0632,  0.2438,
          0.0336,  0.0491],
        [-0.0153, -0.2203,  0.0986, -0.0422, -0.2126, -0.0575,  0.0433,  0.2578,
          0.0315,  0.1222]], grad_fn=<AddmmBackward0>)

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        # F.relu为ReLU的函数版本
        return self.out(F.relu(self.hidden(X)))

In [5]:
net = MLP()
net(X)

tensor([[ 0.0759,  0.0591, -0.1228,  0.1453, -0.0184, -0.1284,  0.1972,  0.0115,
         -0.2993, -0.0092],
        [ 0.1383,  0.0633, -0.1254,  0.2413, -0.0308, -0.0319,  0.1411, -0.1032,
         -0.3522,  0.0543]], grad_fn=<AddmmBackward0>)

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [7]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1156, -0.0285,  0.0500, -0.0479, -0.0908,  0.1701, -0.0780, -0.0909,
          0.1755, -0.1716],
        [ 0.1174,  0.0200, -0.0901, -0.0643, -0.1929,  0.1669,  0.0585, -0.2173,
          0.1892, -0.0724]], grad_fn=<AddmmBackward0>)

In [8]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数，在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)  # 服用全连接层，相当于2个全连接层共享参数
        # 控制流
        while X.abs().sum() > 1:
            X/=2
        return X.sum()

In [10]:
net = FixedHiddenMLP()
net(X)

tensor(-0.0399, grad_fn=<SumBackward0>)

In [11]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0464, grad_fn=<SumBackward0>)